# Obtaining BusStop and MRTStop information
### –––––––––––––––––––––––––––––––––––––––––––––––––––––––––

### Using Geopandas to Import BusStops

In [1]:
import geopandas as gpd
import pandas as pd

ModuleNotFoundError: No module named 'geopandas'

In [ ]:
#Load into GeoDataFrame
gdf_BusStop = gpd.read_file('BusStop.shp')

In [ ]:
gdf_BusStop.head()

In [ ]:
gdf_BusStop.plot()

### Using Geopandas to Import Subzone

In [ ]:
#Load into GeoDataFrame
gdf_Subzone = gpd.read_file('MP14_SUBZONE_WEB_PL.shp')

In [ ]:
gdf_Subzone.head()

### Left join BusStops with Subzone

In [ ]:
from geopandas.tools import sjoin
join_left_df_BusStop = sjoin(gdf_BusStop, gdf_Subzone, how="left")
join_left_df_BusStop

### Filter out relevant rows for BusStop_Subzone

In [ ]:
BusStop_Subzone = join_left_df_BusStop.filter(['BUS_STOP_N','SUBZONE_N','PLN_AREA_N', 'REGION_N'], axis=1)
BusStop_Subzone

### Using Geopandas to Import MRT Stops

In [ ]:
#Load into GeoDataFrame
gdf_MRTStop = gpd.read_file('MRTLRTStnPtt.shp')

In [ ]:
gdf_MRTStop.head()


In [ ]:
gdf_MRTStop.plot()

### Left join MRTStops with Subzone

In [ ]:
from geopandas.tools import sjoin
join_left_df_MRTStop = sjoin(gdf_MRTStop, gdf_Subzone, how="left")
join_left_df_MRTStop
# Note the NaNs where the point did not intersect a boro

### Filter out relevant rows for MRTStop_Subzone

In [ ]:
MRTStop_Subzone = join_left_df_MRTStop.filter(['STN_NAME', 'STN_NO', 'SUBZONE_N', 'PLN_AREA_N', 'REGION_N'], axis=1)
MRTStop_Subzone

# Combine BusStop / MRTStop with Subzone Data (Shihan)
### –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

In [ ]:
dT2020raw = pd.read_excel('singapore-residents-by-planning-areasubzone-age-group-sex-and-type-of-dwelling-june-20002020.xlsx', header = 2, sheet_name = '2020(Total)')
dT2020raw = dT2020raw.rename(columns={'Subzone': 'SUBZONE_N'})
dT2020raw['SUBZONE_N'] = dT2020raw['SUBZONE_N'].str.upper()
dT2020raw = dT2020raw[:69840]
dT2020raw.head(10)

In [ ]:
dT2020 = dT2020raw.filter(['SUBZONE_N', 'Age Group', 'Type of Dwelling', '2020'], axis=1)
MRT_relevant_subzones = MRTStop_Subzone.SUBZONE_N.unique()
print(MRT_relevant_subzones)
dT2020final = dT2020[dT2020['SUBZONE_N'].isin(MRT_relevant_subzones)]
combined = MRTStop_Subzone.merge(MRTStop_Subzone, dT2020final, on='SUBZONE_N', validate='many_to_many')

# Combining Tap In/Tap Out Data into a total "Traffic" column (jeremy)
### –––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

In [ ]:
#//todo yet to finish i think

In [ ]:
BusData202101 = pd.read_csv('2.5_transport_node_bus_202101.csv')
BusData202101.head()

In [ ]:
BusData202101['TOTAL_TAP_IN_TAP_OUT_VOLUME'] = BusData202101['TOTAL_TAP_IN_VOLUME'] + BusData202101['TOTAL_TAP_OUT_VOLUME']
BusData202101Weekday = pd.DataFrame(BusData202101[BusData202101['DAY_TYPE'] == 'WEEKDAY'])
BusData202101WeekendPH = pd.DataFrame(BusData202101[BusData202101['DAY_TYPE'] == 'WEEKENDS/HOLIDAY'])
BusData202101Weekday = BusData202101Weekday.rename(columns = {'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_WEEKDAY', 'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_WEEKDAY', 'TOTAL_TAP_IN_TAP_OUT_VOLUME': 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKDAY'}, inplace = False)
BusData202101WeekendPH = BusData202101WeekendPH.rename(columns = {'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_WEEKEND', 'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_WEEKEND', 'TOTAL_TAP_IN_TAP_OUT_VOLUME': 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND'}, inplace = False)
BusData202101WeekendPH = BusData202101WeekendPH[['DAY_TYPE', 'TIME_PER_HOUR', 'PT_CODE', 'TOTAL_TAP_IN_VOLUME_WEEKEND', 'TOTAL_TAP_OUT_VOLUME_WEEKEND', 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND']]
BusData202101Merge = pd.merge(BusData202101Weekday, BusData202101WeekendPH, on=['PT_CODE', 'TIME_PER_HOUR'])
BusData202101Merge.head()

In [ ]:
BusData202101Merge['TOTAL_TAP_IN_VOLUME'] = BusData202101Merge['TOTAL_TAP_IN_VOLUME_WEEKDAY'] + BusData202101Merge['TOTAL_TAP_IN_VOLUME_WEEKEND']
BusData202101Merge['TOTAL_TAP_OUT_VOLUME'] = BusData202101Merge['TOTAL_TAP_OUT_VOLUME_WEEKDAY'] + BusData202101Merge['TOTAL_TAP_OUT_VOLUME_WEEKEND']
BusData202101Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME'] = BusData202101Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKDAY'] + BusData202101Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND']
BusData202101Merge = BusData202101Merge[['YEAR_MONTH', 'TIME_PER_HOUR', 'PT_CODE', 'TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME', 'TOTAL_TAP_IN_TAP_OUT_VOLUME']]
BusData202101Sorted = BusData202101Merge.sort_values(by=['PT_CODE', 'TIME_PER_HOUR'])
BusData202101Sorted.head()

In [ ]:
BusData202102 = pd.read_csv('2.5_transport_node_bus_202102.csv')
BusData202102.head()

In [ ]:
BusData202102['TOTAL_TAP_IN_TAP_OUT_VOLUME'] = BusData202102['TOTAL_TAP_IN_VOLUME'] + BusData202102['TOTAL_TAP_OUT_VOLUME']
BusData202102Weekday = pd.DataFrame(BusData202102[BusData202102['DAY_TYPE'] == 'WEEKDAY'])
BusData202102WeekendPH = pd.DataFrame(BusData202102[BusData202102['DAY_TYPE'] == 'WEEKENDS/HOLIDAY'])
BusData202102Weekday = BusData202102Weekday.rename(columns = {'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_WEEKDAY', 'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_WEEKDAY', 'TOTAL_TAP_IN_TAP_OUT_VOLUME': 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKDAY'}, inplace = False)
BusData202102WeekendPH = BusData202102WeekendPH.rename(columns = {'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_WEEKEND', 'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_WEEKEND', 'TOTAL_TAP_IN_TAP_OUT_VOLUME': 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND'}, inplace = False)
BusData202102WeekendPH = BusData202102WeekendPH[['DAY_TYPE', 'TIME_PER_HOUR', 'PT_CODE', 'TOTAL_TAP_IN_VOLUME_WEEKEND', 'TOTAL_TAP_OUT_VOLUME_WEEKEND', 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND']]
BusData202102Merge = pd.merge(BusData202102Weekday, BusData202102WeekendPH, on=['PT_CODE', 'TIME_PER_HOUR'])
BusData202102Merge.head()

In [ ]:
BusData202102Merge['TOTAL_TAP_IN_VOLUME'] = BusData202102Merge['TOTAL_TAP_IN_VOLUME_WEEKDAY'] + BusData202102Merge['TOTAL_TAP_IN_VOLUME_WEEKEND']
BusData202102Merge['TOTAL_TAP_OUT_VOLUME'] = BusData202102Merge['TOTAL_TAP_OUT_VOLUME_WEEKDAY'] + BusData202102Merge['TOTAL_TAP_OUT_VOLUME_WEEKEND']
BusData202102Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME'] = BusData202102Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKDAY'] + BusData202102Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND']
BusData202102Merge = BusData202102Merge[['YEAR_MONTH', 'TIME_PER_HOUR', 'PT_CODE', 'TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME', 'TOTAL_TAP_IN_TAP_OUT_VOLUME']]
BusData202102Sorted = BusData202102Merge.sort_values(by=['PT_CODE', 'TIME_PER_HOUR'])
BusData202102Sorted.head()

In [ ]:
BusData202103 = pd.read_csv('2.5_transport_node_bus_202103.csv')
BusData202103.head()

In [ ]:
BusData202103['TOTAL_TAP_IN_TAP_OUT_VOLUME'] = BusData202103['TOTAL_TAP_IN_VOLUME'] + BusData202103['TOTAL_TAP_OUT_VOLUME']
BusData202103Weekday = pd.DataFrame(BusData202103[BusData202103['DAY_TYPE'] == 'WEEKDAY'])
BusData202103WeekendPH = pd.DataFrame(BusData202103[BusData202103['DAY_TYPE'] == 'WEEKENDS/HOLIDAY'])
BusData202103Weekday = BusData202103Weekday.rename(columns = {'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_WEEKDAY', 'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_WEEKDAY', 'TOTAL_TAP_IN_TAP_OUT_VOLUME': 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKDAY'}, inplace = False)
BusData202103WeekendPH = BusData202103WeekendPH.rename(columns = {'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_WEEKEND', 'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_WEEKEND', 'TOTAL_TAP_IN_TAP_OUT_VOLUME': 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND'}, inplace = False)
BusData202103WeekendPH = BusData202103WeekendPH[['DAY_TYPE', 'TIME_PER_HOUR', 'PT_CODE', 'TOTAL_TAP_IN_VOLUME_WEEKEND', 'TOTAL_TAP_OUT_VOLUME_WEEKEND', 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND']]
BusData202103Merge = pd.merge(BusData202103Weekday, BusData202103WeekendPH, on=['PT_CODE', 'TIME_PER_HOUR'])
BusData202103Merge.head()

In [ ]:
BusData202103Merge['TOTAL_TAP_IN_VOLUME'] = BusData202103Merge['TOTAL_TAP_IN_VOLUME_WEEKDAY'] + BusData202103Merge['TOTAL_TAP_IN_VOLUME_WEEKEND']
BusData202103Merge['TOTAL_TAP_OUT_VOLUME'] = BusData202103Merge['TOTAL_TAP_OUT_VOLUME_WEEKDAY'] + BusData202103Merge['TOTAL_TAP_OUT_VOLUME_WEEKEND']
BusData202103Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME'] = BusData202103Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKDAY'] + BusData202103Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND']
BusData202103Merge = BusData202103Merge[['YEAR_MONTH', 'TIME_PER_HOUR', 'PT_CODE', 'TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME', 'TOTAL_TAP_IN_TAP_OUT_VOLUME']]
BusData202103Sorted = BusData202103Merge.sort_values(by=['PT_CODE', 'TIME_PER_HOUR'])
BusData202103Sorted.head()

In [ ]:
MRTData202101 = pd.read_csv('2.8_transport_node_train_202101.csv')
MRTData202101.head()

In [ ]:
MRTData202101['TOTAL_TAP_IN_TAP_OUT_VOLUME'] = MRTData202101['TOTAL_TAP_IN_VOLUME'] + MRTData202101['TOTAL_TAP_OUT_VOLUME']
MRTData202101Weekday = pd.DataFrame(MRTData202101[MRTData202101['DAY_TYPE'] == 'WEEKDAY'])
MRTData202101WeekendPH = pd.DataFrame(MRTData202101[MRTData202101['DAY_TYPE'] == 'WEEKENDS/HOLIDAY'])
MRTData202101Weekday = MRTData202101Weekday.rename(columns = {'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_WEEKDAY', 'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_WEEKDAY', 'TOTAL_TAP_IN_TAP_OUT_VOLUME': 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKDAY'}, inplace = False)
MRTData202101WeekendPH = MRTData202101WeekendPH.rename(columns = {'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_WEEKEND', 'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_WEEKEND', 'TOTAL_TAP_IN_TAP_OUT_VOLUME': 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND'}, inplace = False)
MRTData202101WeekendPH = MRTData202101WeekendPH[['DAY_TYPE', 'TIME_PER_HOUR', 'PT_CODE', 'TOTAL_TAP_IN_VOLUME_WEEKEND', 'TOTAL_TAP_OUT_VOLUME_WEEKEND', 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND']]
MRTData202101Merge = pd.merge(MRTData202101Weekday, MRTData202101WeekendPH, on=['PT_CODE', 'TIME_PER_HOUR'])
MRTData202101Merge.head()

In [ ]:
MRTData202101Merge['TOTAL_TAP_IN_VOLUME'] = MRTData202101Merge['TOTAL_TAP_IN_VOLUME_WEEKDAY'] + MRTData202101Merge['TOTAL_TAP_IN_VOLUME_WEEKEND']
MRTData202101Merge['TOTAL_TAP_OUT_VOLUME'] = MRTData202101Merge['TOTAL_TAP_OUT_VOLUME_WEEKDAY'] + MRTData202101Merge['TOTAL_TAP_OUT_VOLUME_WEEKEND']
MRTData202101Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME'] = MRTData202101Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKDAY'] + MRTData202101Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND']
MRTData202101Merge = MRTData202101Merge[['YEAR_MONTH', 'TIME_PER_HOUR', 'PT_CODE', 'TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME', 'TOTAL_TAP_IN_TAP_OUT_VOLUME']]
MRTData202101Sorted = MRTData202101Merge.sort_values(by=['PT_CODE', 'TIME_PER_HOUR'])
MRTData202101Sorted.head()

In [ ]:
MRTData202102 = pd.read_csv('2.8_transport_node_train_202102.csv')
MRTData202102.head()

In [ ]:
MRTData202102['TOTAL_TAP_IN_TAP_OUT_VOLUME'] = MRTData202102['TOTAL_TAP_IN_VOLUME'] + MRTData202102['TOTAL_TAP_OUT_VOLUME']
MRTData202102Weekday = pd.DataFrame(MRTData202102[MRTData202102['DAY_TYPE'] == 'WEEKDAY'])
MRTData202102WeekendPH = pd.DataFrame(MRTData202102[MRTData202102['DAY_TYPE'] == 'WEEKENDS/HOLIDAY'])
MRTData202102Weekday = MRTData202102Weekday.rename(columns = {'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_WEEKDAY', 'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_WEEKDAY', 'TOTAL_TAP_IN_TAP_OUT_VOLUME': 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKDAY'}, inplace = False)
MRTData202102WeekendPH = MRTData202102WeekendPH.rename(columns = {'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_WEEKEND', 'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_WEEKEND', 'TOTAL_TAP_IN_TAP_OUT_VOLUME': 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND'}, inplace = False)
MRTData202102WeekendPH = MRTData202102WeekendPH[['DAY_TYPE', 'TIME_PER_HOUR', 'PT_CODE', 'TOTAL_TAP_IN_VOLUME_WEEKEND', 'TOTAL_TAP_OUT_VOLUME_WEEKEND', 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND']]
MRTData202102Merge = pd.merge(MRTData202102Weekday, MRTData202102WeekendPH, on=['PT_CODE', 'TIME_PER_HOUR'])
MRTData202102Merge.head()

In [ ]:
MRTData202102Merge['TOTAL_TAP_IN_VOLUME'] = MRTData202102Merge['TOTAL_TAP_IN_VOLUME_WEEKDAY'] + MRTData202102Merge['TOTAL_TAP_IN_VOLUME_WEEKEND']
MRTData202102Merge['TOTAL_TAP_OUT_VOLUME'] = MRTData202102Merge['TOTAL_TAP_OUT_VOLUME_WEEKDAY'] + MRTData202102Merge['TOTAL_TAP_OUT_VOLUME_WEEKEND']
MRTData202102Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME'] = MRTData202102Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKDAY'] + MRTData202102Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND']
MRTData202102Merge = MRTData202102Merge[['YEAR_MONTH', 'TIME_PER_HOUR', 'PT_CODE', 'TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME', 'TOTAL_TAP_IN_TAP_OUT_VOLUME']]
MRTData202102Sorted = MRTData202102Merge.sort_values(by=['PT_CODE', 'TIME_PER_HOUR'])
MRTData202102Sorted.head()

In [ ]:
MRTData202103 = pd.read_csv('2.8_transport_node_train_202103.csv')
MRTData202103.head()

In [ ]:
MRTData202103['TOTAL_TAP_IN_TAP_OUT_VOLUME'] = MRTData202103['TOTAL_TAP_IN_VOLUME'] + MRTData202103['TOTAL_TAP_OUT_VOLUME']
MRTData202103Weekday = pd.DataFrame(MRTData202103[MRTData202103['DAY_TYPE'] == 'WEEKDAY'])
MRTData202103WeekendPH = pd.DataFrame(MRTData202103[MRTData202103['DAY_TYPE'] == 'WEEKENDS/HOLIDAY'])
MRTData202103Weekday = MRTData202103Weekday.rename(columns = {'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_WEEKDAY', 'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_WEEKDAY', 'TOTAL_TAP_IN_TAP_OUT_VOLUME': 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKDAY'}, inplace = False)
MRTData202103WeekendPH = MRTData202103WeekendPH.rename(columns = {'TOTAL_TAP_IN_VOLUME': 'TOTAL_TAP_IN_VOLUME_WEEKEND', 'TOTAL_TAP_OUT_VOLUME': 'TOTAL_TAP_OUT_VOLUME_WEEKEND', 'TOTAL_TAP_IN_TAP_OUT_VOLUME': 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND'}, inplace = False)
MRTData202103WeekendPH = MRTData202103WeekendPH[['DAY_TYPE', 'TIME_PER_HOUR', 'PT_CODE', 'TOTAL_TAP_IN_VOLUME_WEEKEND', 'TOTAL_TAP_OUT_VOLUME_WEEKEND', 'TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND']]
MRTData202103Merge = pd.merge(MRTData202103Weekday, MRTData202103WeekendPH, on=['PT_CODE', 'TIME_PER_HOUR'])
MRTData202103Merge.head()

In [ ]:
MRTData202103Merge['TOTAL_TAP_IN_VOLUME'] = MRTData202103Merge['TOTAL_TAP_IN_VOLUME_WEEKDAY'] + MRTData202103Merge['TOTAL_TAP_IN_VOLUME_WEEKEND']
MRTData202103Merge['TOTAL_TAP_OUT_VOLUME'] = MRTData202103Merge['TOTAL_TAP_OUT_VOLUME_WEEKDAY'] + MRTData202103Merge['TOTAL_TAP_OUT_VOLUME_WEEKEND']
MRTData202103Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME'] = MRTData202103Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKDAY'] + MRTData202103Merge['TOTAL_TAP_IN_TAP_OUT_VOLUME_WEEKEND']
MRTData202103Merge = MRTData202103Merge[['YEAR_MONTH', 'TIME_PER_HOUR', 'PT_CODE', 'TOTAL_TAP_IN_VOLUME', 'TOTAL_TAP_OUT_VOLUME', 'TOTAL_TAP_IN_TAP_OUT_VOLUME']]
MRTData202103Sorted = MRTData202103Merge.sort_values(by=['PT_CODE', 'TIME_PER_HOUR'])
MRTData202103Sorted.head()

# Representing Categorical Data with the Use of Dummy Variables (bleow)
### ––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––––

In [ ]:
#predictorss = pd.get_dummies(data=predictors, drop_first=True)
#predictorss

## In creating the dummy variables, the drop_first function will help us prevent structural multicollinearity.

### Multicollinearity?
Multicollinearity is a state of very high correlation among the independent variables, i.e. A predictor variable can be used to predict another predictor variable.

### Structural multicollinearity?
Structural multicollinearity occurs when we create a model term using other terms. In other words, it’s a byproduct of the model that we specify rather than being present in the data itself. For example, if you square term X to model curvature, clearly there is a correlation between X and X2.

### Why does it affect the regression?
Both the independent features have a similar impact on the dependent variable so the regression model fails to understand the individual effect of each independent variables on the dependent variable.

For dummy variables, 1 feature among the 4 can be calculated without having to make a separate variable.

### Solution
We make use of drop_first parameter while creating dummy to drop the last variable from the table to prevent this multicollinearity. We will call the dropped variables the baseline. Which were the baseline categories used?

In [ ]:
#predictorsDiff = (pd.get_dummies(data=predictors) - predictorss) # Perform datacell-wise subtraction of data. The result will be a 0 for data that exists in both dataframes
#predictorsDiff = predictorsDiff.loc[:, ~(predictorsDiff == 0).all()] # Select columns that do NOT contain data where it is all =0. In this case, this means to extract the columns that exist when drop_first is False and do not exist when drop_first is True.
#baselines = predictorsDiff.columns
#baselines

# Performing the LINEAR REGRESSION
### –––––––––––––––––––––––––––––––––––––––––––––––

In [ ]:
'''# Import essential models and functions from sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Split the Dataset into Train and Test
X_train, X_test, y_train, y_test = train_test_split(predictorss, response, test_size = 0.25)

# Check the sample sizes
print('Train Set :', y_train.shape, X_train.shape)
print('Test Set  :', y_test.shape, X_test.shape)'''

In [ ]:
'''# Linear regression using train data
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Coefficients of the Linear Regression line
#print('Intercept of Regression \t: b = ', linreg.intercept_)
#print('Coefficients of Regression \t: a = ', linreg.coef_)
#print()

coeff = pd.DataFrame(list(zip(X_train.columns, linreg.coef_[0])), columns = ["Predictors", "Coefficients"])
coeff = coeff.sort_values(by='Coefficients', ascending=False)

with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    # Print the Coefficients against Predictors
    print(coeff)'''

In [ ]:
'''# Predict the Total values from Predictors
y_train_pred = linreg.predict(X_train)
y_test_pred = linreg.predict(X_test)

# Plot the Predictions vs the True values
f, axes = plt.subplots(1, 2, figsize=(24, 12))
axes[0].scatter(y_train, y_train_pred, color = "blue")
axes[0].plot(y_train, y_train, 'w-', linewidth = 1)
axes[0].set_xlabel("True values of the Response Variable (Train)")
axes[0].set_ylabel("Predicted values of the Response Variable (Train)")
axes[1].scatter(y_test, y_test_pred, color = "green")
axes[1].plot(y_test, y_test, 'w-', linewidth = 1)
axes[1].set_xlabel("True values of the Response Variable (Test)")
axes[1].set_ylabel("Predicted values of the Response Variable (Test)")
plt.show()'''

In [ ]:
'''# Check the Goodness of Fit (on Train Data)
print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", linreg.score(X_train, y_train))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_train, y_train_pred))
print()

# Check the Goodness of Fit (on Test Data)
print("Goodness of Fit of Model \tTest Dataset")
print("Explained Variance (R^2) \t:", linreg.score(X_test, y_test))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test, y_test_pred))
print()'''